# Extract Drawing ROI
This notebook extracts the coordinates of the drawing using the output of the `fastroi` processing. The output is a CSV file with each image path and the ROI of the drawing within the image. The ROI is computed given the estimated XY center by setting a box around it of 1.5x the size of the template height and width.

In [1]:
%matplotlib inline

import math
from pathlib import Path
from typing import Dict, List

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from bananas.utils import images

# Root path of project relative to this notebook
ROOT = Path('..')

sys.path.insert(1, str(ROOT / 'scripts'))
from datamodels import *
from utils import *

### Load all templates from the templates folder

In [2]:
templates = tpl_load_all(ROOT / 'templates')
[tpl.name for tpl in templates]

['casa',
 'circulo',
 'minimental',
 'pico',
 'cruz',
 'muelle',
 'cubo',
 'cuadrado',
 'triangulo']

### Extract the coordinates from the individual CSV files in the `fastroi` folder

In [3]:
def roi_drawing_map(tpl: Template, fastroi_dir: Path, img_root: Path, scaling: float = 1.5) -> Dict[str, Box]:
    ''' Reads the drawing CSV file from `fastroi` and puts it into a map with the name as a key '''
    roi_map = {}
    for line in open(fastroi_dir / ('%s.csv' % tpl.name)):
        fname, coords = line.split(',', 1)
        fname = fname.split('/')[-1]
        coords = list(map(int, coords.split(',')))
        img = images.open_image(str(img_root / fname))
        roi_map[fname] = coords_to_roi(coords, tpl, img.shape, scaling=scaling)
    return roi_map

def coords_to_roi(coords: List[int], tpl: Template, img_size: List[int], scaling: float):
    ''' Converts XY center coordinates into a box assuming similar size compared to template'''
    
    # Open image to determine height and width
    y_max, x_max = img_size[0], img_size[1]
    
    # Estimate the halves of the resulting ROI in whole pixels
    w_max_half = int(tpl.width * scaling / 2)
    h_max_half = int(tpl.height * scaling / 2)
    
    return Box(
        left = max(0, coords[0] - w_max_half),
        top = max(0, coords[1] - h_max_half),
        right = min(x_max, coords[0] + w_max_half),
        bottom = min(y_max, coords[1] + h_max_half))


In [4]:
# Iterate over all image drawings
drawing_roi_records = []
for tpl in tqdm(templates):
    imdir = ROOT / 'drawings' / tpl.name.upper()
    impaths = [impath for impath in imdir.iterdir() if impath.suffix in IMG_EXTENSIONS]
    
    # Extract drawing ROIs from our map
    roi_map = roi_drawing_map(tpl, ROOT / 'fastroi', imdir)
    
    # Iterate over each image and append record with the corresponding ROI
    for impath in tqdm(impaths, leave=False):
        imname = impath.name
        relpath = str(impath.relative_to(ROOT))
        drawing_roi = roi_map.get(imname)
        if drawing_roi:
            drawing_roi_records.append({
                'image_path': str(relpath),
                'drawing_box': str(drawing_roi)})
            
print(len(drawing_roi_records), 'ROIs processed')


6190 ROIs processed


### Write results in a CSV file

In [5]:
pd.DataFrame.from_records(drawing_roi_records).set_index('image_path').to_csv(ROOT / 'datasets' / 'drawing_roi.csv')